# Fake News Classifier using LSTM

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
df = pd.read_csv("fake-news/train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df = df.dropna()

In [5]:
X = df.drop("label", axis=1)
y = df["label"]

In [6]:
voc_size = 5000

In [7]:
messages = X.copy()
messages.reset_index(inplace=True)

## Data Preprocessing

In [8]:
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    review = re.sub("[^a-zA-Z]", " ", messages["title"][i]).lower().split()
    review = [ps.stem(word) for word in review if not word in stopwords.words("english")]
    review = " ".join(review)
    corpus.append(review)

In [9]:
corpus[:5]

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

## One Hot Representation

In [10]:
onehot_repr = [one_hot(words, voc_size) for words in corpus]
onehot_repr[:5]

[[3065, 1853, 258, 272, 3277, 4602, 763, 88, 4949, 2966],
 [2903, 3525, 1956, 4845, 1952, 3836, 4607],
 [1102, 3121, 1501, 2538],
 [585, 2049, 1067, 506, 4460, 1196],
 [164, 1952, 2615, 4435, 4880, 3849, 1952, 170, 2788, 1171]]

## Embedding Representation

In [11]:
sent_len = 20
embedded_doc = pad_sequences(onehot_repr, padding='pre', maxlen=sent_len)
embedded_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3065,
       1853,  258,  272, 3277, 4602,  763,   88, 4949, 2966])

## Creating model

In [12]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size,embedding_vector_features, input_length=sent_len))
model.add(LSTM(100))
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
X_final = np.array(embedded_doc)
y_final = np.array(y)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [15]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 6s 23ms/step - loss: 0.3424 - accuracy: 0.8380 - val_loss: 0.1918 - val_accuracy: 0.9140
Epoch 2/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1345 - accuracy: 0.9461 - val_loss: 0.2049 - val_accuracy: 0.9198
Epoch 3/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0940 - accuracy: 0.9669 - val_loss: 0.2133 - val_accuracy: 0.9167
Epoch 4/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0625 - accuracy: 0.9798 - val_loss: 0.2658 - val_accuracy: 0.9075
Epoch 5/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0384 - accuracy: 0.9873 - val_loss: 0.3052 - val_accuracy: 0.9132
Epoch 6/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0221 - accuracy: 0.9933 - val_loss: 0.5007 - val_accuracy: 0.9135
Epoch 7/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0150 - accuracy: 0.9962 - val_loss: 0.4658 - val_accuracy: 0.9157

## Performance Metrics

In [16]:
y_pred = (model.predict(X_test) > 0.5).astype("int32")

189/189 [==============================] - 1s 5ms/step


In [17]:
confusion_matrix(y_test, y_pred)

array([[3103,  316],
       [ 221, 2395]], dtype=int64)

In [18]:
accuracy_score(y_test, y_pred)

0.9110190555095278